In [166]:
import time
import json
import tweepy


In [197]:

API_KEYS = [
    
    {
        "API_KEY": 'ZhrmtUABV2zVZDccjgUjX5j6Y',
        "API_SECRET": 'fLqx4LgROPuz5y0zZkhyRWR9nvOGjwhlqkASxFj3XfIINHtLeA',
        "BEARER_TOKEN" :'AAAAAAAAAAAAAAAAAAAAAL2qxAEAAAAAStCSHTdKBoLvlyQm37MG5hm6ag4%3DJTSx8uj6vlieIZz2XC6i9TJSkpdTO3EzLjVfkOsq0OBAUG3sUn',
    },
    
    # {
    #     "API_KEY": "SpCFoPDtLOC6rTo688No0FG3I",
    #     "API_SECRET": "aOWpWsk7jp5eGgzouQvHNIjcIcOmhSMbIbUNBUtwWw8CrzpUG8",
    #     "BEARER_TOKEN": "AAAAAAAAAAAAAAAAAAAAABWXxAEAAAAARFv6fUtIQy04V93TqqXncv1UKmM%3DE9qE2OOg18LCYt0AmzzzoraONdhHKxpKZObRhrEQCk5Xx0UB5F",
    # },

]

## Top topics
- AI art ethical （done）
- AI Killing Creativity （done）
- AI Art Copyright (done) 70
- AI Art Exploitation (done) 30
- AI Art Fairness (done) 1
- AI Art Soulless (done) 96
- AI Art Theft (done) 99

In [200]:
query = "AI Art Theft" 
max_results = 100  
total_tweets_to_fetch = 1000  
pause_time = 30  


In [201]:

tweets = []  
next_token = None  
current_api_index = 0  
rate_limit_count = 0 
total_rate_limit_count = 0 

# Capture theme-related tweets
while len(tweets) < total_tweets_to_fetch:
    try:
        # Change current API key
        api = API_KEYS[current_api_index]
        client = tweepy.Client(bearer_token=api["BEARER_TOKEN"])
        print(f"Using API key {current_api_index + 1} grab themed tweets...")

        # Using API 
        response = client.search_recent_tweets(
            query=query,
            max_results=max_results,
            next_token=next_token
        )


        # if there has data, saving it to the list
        if response.data:
            tweets.extend(response.data)
            print(f"Currently, a total of {len(tweets)} topic tweets have been captured.")

        else:
            print("No more tweets to capture.")

            

        # Get paging token
        next_token = response.meta.get("next_token")
        if not next_token:
            print(f"API key {current_api_index + 1} 已完成抓取。")
            current_api_index += 1  # change it to next API key
            next_token = None  # Reset paging token
            if current_api_index >= len(API_KEYS):  
                print("All API keys have been used up.")

    except tweepy.TooManyRequests:
        # Handle rate limit
        print(f"API key {current_api_index + 1} has hit the rate limit, pausing for {pause_time // 60} minutes...")
        time.sleep(pause_time)
        rate_limit_count += 1  # Count the rate limit occurrences
        total_rate_limit_count += 1  # Update total rate limit count

        # If all API keys hit the rate limit, skip comment fetching and end fetching
        if total_rate_limit_count >= len(API_KEYS) * 3:
            print("All API keys have reached the rate limit, skipping comment fetching and saving data directly...")
            break

    except Exception as e:
        # Handle other errors
        print(f"An error occurred: {e}")
        break


# remove the duplicates
unique_tweets = list({tweet.id: tweet for tweet in tweets}.values())


# make it to json files
tweet_data = [{"id": tweet.id, "text": tweet.text, "created_at": str(tweet.created_at)} for tweet in unique_tweets]


# saving the files
with open("tweets.json", "w", encoding="utf-8") as f:
    json.dump(tweet_data, f, ensure_ascii=False, indent=4)

print(f"A total of {len(unique_tweets)} topic tweets have been captured.")



Using API key 1 grab themed tweets...
Currently, a total of 99 topic tweets have been captured.
Using API key 1 grab themed tweets...
API key 1 has hit the rate limit, pausing for 0 minutes...
Using API key 1 grab themed tweets...
API key 1 has hit the rate limit, pausing for 0 minutes...
Using API key 1 grab themed tweets...
API key 1 has hit the rate limit, pausing for 0 minutes...
All API keys have reached the rate limit, skipping comment fetching and saving data directly...
A total of 99 topic tweets have been captured.


In [204]:
import json
import re
import pandas as pd

# Function to clean text
def clean_text(text):
    # 1. Remove extra spaces
    text = text.strip()
    
    # 2. Remove newline characters and extra spaces
    text = re.sub(r'\s+', ' ', text)  # Replace multiple whitespace characters (including newline, tabs) with a single space
    
    # 3. Remove meaningless characters, such as special symbols (can be adjusted as needed)
    text = re.sub(r'[^\w\s,.\-]', '', text)  # Keep letters, numbers, spaces, commas, periods, and hyphens
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove links
    text = re.sub(r'@\S+', '', text)  # Remove @username mentions
    
    # 4. Optional: Convert the text to lowercase
    text = text.lower()
    text = text.replace('"', '')  # Remove double quotes

    if text.startswith('"') and text.endswith('"'):
        text = text[1:-1]

    # 5. If you want to remove tags (e.g., # or @), you can add this step
    text = re.sub(r'#[\w]+', '', text)  # Remove all hashtags (e.g., #aiart)
    text = re.sub(r'@\S+', '', text)    # Remove all usernames (e.g., @ai_ethics)
    text = re.sub(r'-', '', text) 
    return text

def remove_non_english_words(text):
    # English words
    english_words = re.findall(r'\b[a-zA-Z]+\b', text)
    return ' '.join(english_words)

# Open and read the original data file
data = []
with open('cleaned_tweets.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Clean the text of each tweet
for tweet in data:
    tweet['text'] = clean_text(tweet['text'])  # Clean the 'text' field
    tweet['text'] = remove_non_english_words(tweet['text'])

# Use pandas to convert the data into a DataFrame
df = pd.DataFrame(data)

# Drop the 'created_at' column
df = df.drop(columns=['created_at'])

# Drop duplicate rows based on the 'text' column
df = df.drop_duplicates(subset=['text'])

# Output the DataFrame to a CSV file
df.to_csv('all_data.csv', index=False, encoding='utf-8')

print("Data cleaned and written to all_data.csv")





Data cleaned and written to all_data.csv
